In [1]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation

%matplotlib inline 
sns.set(color_codes=True)

### Import filtered Student's file 

In [2]:
# read in the .csv file - of filtered student_info

data_df = pd.read_csv("filtered_df2_A2.csv")

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_student         383 non-null    int64 
 1   gender             383 non-null    int64 
 2   highest_education  383 non-null    object
 3   age_band           383 non-null    object
 4   studied_credits    383 non-null    int64 
 5   final_result       383 non-null    object
 6   tenure_band        383 non-null    object
dtypes: int64(3), object(4)
memory usage: 21.1+ KB


### Encoding categorical data

Using One-Hot Encoding

In [4]:
# One-Hot Encoding for 'highest_education', 'age_band', 'final_result', 'tenure_band'

data_df_encoded = pd.get_dummies(data_df, columns=['highest_education', 'age_band', 'final_result', 'tenure_band'])

In [5]:
#data_df_encoded.info()

In [6]:
# convert the dtype of the recently one-hot encoded columns to int64 from uint8 dtype

columns_to_convert = [
    'highest_education_A Level or Equivalent',
    'highest_education_HE Qualification',
    'highest_education_Lower Than A Level',
    'highest_education_Post Graduate Qualification',
    'age_band_0-35',
    'age_band_35-55',
    'age_band_55<=',
    'final_result_Distinction',
    'final_result_Fail',
    'final_result_Pass',
    'final_result_Withdrawn', 
    'tenure_band_0-5',
    'tenure_band_11-15',
    'tenure_band_16 or more',
    'tenure_band_5-10'
]

# Convert the selected columns to int64 dtype
data_df_encoded[columns_to_convert] = data_df_encoded[columns_to_convert].astype('int64')

In [7]:
#data_df_encoded.info()

### Algorithm 3 - Support Vector Machine (SVM) 

#### https://scikit-learn.org/stable/modules/svm.html

In [8]:
# import libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [9]:
# divide the dataset into features (X) and the target variable (y)

X = data_df_encoded.drop(columns=['studied_credits'])  # Replace 'target_variable' with the name of your target variable column

y = data_df_encoded['studied_credits']

In [10]:
# split the data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# feature scaling (recommended for SVM)

scaler = StandardScaler()

In [12]:
# scale x dataframes - test and train
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# create the SVM model
# kernels to try ... 'linear', 'rbf' (default), 'poly', 'sigmoid', 'callable'/''

svm_model = SVC(kernel='rbf')  

In [14]:
# train the SVM model on the scaled training data

#svm_model.fit(X_train_scaled, y_train)

#### Hyper parameter tuning

In [15]:
# defining the hyperparameter grid to search

param_grid = {
    # Regularization parameter
    'C': [0.1, 1, 10, 100],       
    
    # Kernel coefficient for 'rbf'
    'gamma': [0.001, 0.01, 0.1],  
    
    # Different kernels to explore - v2 of gridsearch  CV
    'kernel': ['linear', 'poly', 'rbf']  
}

In [16]:
# create the GridSearchCV model with SVM

grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)


In [17]:
# train the GridSearchCV model on the scaled training data

grid_search.fit(X_train_scaled, y_train)


C:\Users\sinea\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1],
                         'kernel': ['linear', 'poly', 'rbf']})

In [18]:
# find the best hyperparameters from the search

best_params = grid_search.best_params_

In [19]:
# train the SVM model with the best hyperparameters on the entire training data

best_svm_model = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'])

In [20]:
# fit the SVM model to the scaled data

best_svm_model.fit(X_train_scaled, y_train)

SVC(C=0.1, gamma=0.001)

In [21]:
# make predictions on the scaled test data

#y_pred = svm_model.predict(X_test_scaled)

y_pred = best_svm_model.predict(X_test_scaled)

In [22]:
# calculate the accuracy of the model

#accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7012987012987013


In [23]:
# calculate the accuracy of the best model

print("Best Model Accuracy:", accuracy)
print("Best Hyperparameters:", best_params)

Best Model Accuracy: 0.7012987012987013
Best Hyperparameters: {'C': 0.1, 'gamma': 0.001, 'kernel': 'linear'}


#### Plotting the GridSearchCV

In [24]:
# get the results of the hyperparameter search .. only for attempt 2 

results = grid_search.cv_results_

# create a variable to hold the results - for attempt 2
mean_test_scores = np.array(results['mean_test_score']).reshape(len(param_grid['C']), len(param_grid['gamma']))


ValueError: cannot reshape array of size 36 into shape (4,3)

In [ ]:
# attempt 2 of Gridsearch CV with param_grid
# Reshape the mean_test_score array for each kernel separately

mean_test_scores = {}
for kernel in param_grid['kernel']:
    mean_test_scores[kernel] = results['mean_test_score'][results['param_kernel'] == kernel].reshape(len(param_grid['C']), len(param_grid['gamma']))

In [ ]:
# create a heatmap plot for 1st Gridsearch CV code

plt.figure(figsize=(8, 6))
plt.imshow(mean_test_scores, cmap='viridis', origin='lower', aspect='auto',
           extent=[np.min(param_grid['gamma']), np.max(param_grid['gamma']),
                   np.min(param_grid['C']), np.max(param_grid['C'])])
plt.colorbar(label='Mean Test Score')
plt.xlabel('Gamma variable')
plt.ylabel('C variable')
plt.xticks(param_grid['gamma'])
plt.yticks(param_grid['C'])
plt.title('SVM Performance with GridSearchCV')
plt.show()